In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
excelfile = 'CCE-soil BJC.xls'

#soil_df = pd.read_excel(excelfile, sheet_name='CCE-soil', header=16, skiprows=[17], index_col=1)

multi.do_magic_loop(dataset, excelfile, None)

#soil_df

Running lake Hokksjøen (ID 9)

